In [5]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息素材準備
handler執行方法設計
啟動應用


'''

'\n掛載Google硬碟\n安裝套件\n引用套件\nAPP應用準備\n消息素材準備\nhandler執行方法設計\n啟動應用\n\n\n'

In [6]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
'''

流程解析

'''


'\n\n流程解析\n\n'

In [8]:
!pip install line-bot-sdk flask flask-ngrok

In [9]:
'''
引用套件
'''


# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [10]:
'''
建置主程序app

建置handler與 line_bot_api
'''

# 設定Server啟用細節
#準備app這個flask http app
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
#請ngrok給我們一個http臨時url
run_with_ngrok(app)
# 生成實體物件
#跟Line溝通
line_bot_api = LineBotApi("NLuJuE98mu9aVvIIfWPrgfq9q5EoVZYP+yWK6W4O+UsYdS/GmMpUYFk7qEM2OrrDXikIV+ZRTA1V/BftphfFFqpzzA8m+ygvmWsZ7iCn0kxank+IemyqfWLa7kYok8WV5d+3cTUXJC24ndjiGG6nDgdB04t89/1O/w1cDnyilFU=")
#收發消息專用
handler = WebhookHandler("b660a0127b1ab050edfe4f9d9c543b4c")

In [11]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''

# 啟動server對外接口，使Line能丟消息進來
#預設用來讓line都消息的地方
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    #取得line給的簽章(固定)
    signature = request.headers['X-Line-Signature']

    # get request body as text
    #取得用戶消息
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/2021-AI/LineBot/1020event/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    #把消息交給handler做處理
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [12]:
'''

消息素材準備

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)
#xxxSendMessage,回應用戶的消息物件
specific_text_message=TextSendMessage("官方建議，用戶以文字消息觸發特殊功能的時候，要加上特殊符號")
#回應用戶的圖片消息
#參數與line bot designer很類似
#得是https網址(未來可以上傳到cloud storage)
analyze_truth_image_message=ImageSendMessage(
      original_content_url='https://i.imgur.com/M8jnn9B.png',
      preview_image_url='https://i.imgur.com/LtCtgg0.jpg'
      )

In [13]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''
#設定字典,依照字典回應消息
#未來很有可能是資料庫,把你要回應的消息存在資料庫內,用戶回應,從資料庫找
# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  "@123":specific_text_message,
  "@分析的本質": analyze_truth_image_message
}

In [14]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
# @handler.add(Event)
#MessageEvent消息事件
#文字消息 圖片消息(ImageMessage) 影片消息(VideoMessage)等等
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    #當用戶傳來的消息有@時,我們就從字典裡找回應
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    #若無,則回傳查無此字
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@123或@分析的本質")
        )
#TextMessage >用戶傳的
#TextSendMessage >我們傳給用戶的

In [ ]:
# 主程序運行
app.run()
#用戶傳來的文字,建議加上特殊符號

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7b1a-34-80-95-227.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"U5307897c9c8ff435a32a1d50e95ee58e","events":[{"type":"message","message":{"type":"text","id":"14944113910414","text":"@123"},"timestamp":1634710869719,"source":{"type":"user","userId":"Ud28913e45991f8c7531b94f85a9992a1"},"replyToken":"b95f4c000d3641f58d38b70ad1db3455","mode":"active"}]}


127.0.0.1 - - [20/Oct/2021 06:21:11] "POST / HTTP/1.1" 200 -
